# Australia state and territory mask

State boundaries from ABS: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

In [ ]:
import geopandas
import regionmask